<a href="https://colab.research.google.com/github/hila-chefer/Transformer-Explainability/blob/main/BERT_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import visualization
import torch

In [3]:
!pip install ipywidgets

  Using cached ipywidgets-8.1.3-py3-none-any.whl (139 kB)
  Using cached widgetsnbextension-4.0.11-py3-none-any.whl (2.3 MB)
  Using cached jupyterlab_widgets-3.0.11-py3-none-any.whl (214 kB)


In [2]:
model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2").to("cuda")
model.eval()
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")
# initialize the explanations generator
explanations = Generator(model)

classifications = ["NEGATIVE", "POSITIVE"]


#Positive sentiment example

In [3]:
# encode a sentence
text_batch = ["This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('this', 0.4280132055282593), ('movie', 0.31120723485946655), ('was', 0.26965343952178955), ('the', 0.3516356945037842), ('best', 0.6295801401138306), ('movie', 0.28795918822288513), ('i', 0.18626655638217926), ('have', 0.10151874274015427), ('ever', 0.1415516436100006), ('seen', 0.1910468190908432), ('!', 0.594453752040863), ('some', 0.0044118622317910194), ('scenes', 0.03362831473350525), ('were', 0.019243450835347176), ('ridiculous', 0.01820136047899723), (',', 0.0), ('but', 0.43073737621307373), ('acting', 0.43740659952163696), ('was', 0.5009785890579224), ('great', 1.0), ('.', 0.013526408933103085), ('[SEP]', 0.0889810249209404)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),1,1.00,"[CLS] this movie was the best movie i have ever seen ! some scenes were ridiculous , but acting was great . [SEP]"


#Negative sentiment example

In [4]:
# encode a sentence
text_batch = ["I really didn't like this movie. Some of the actors were good, but overall the movie was boring."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                1,
                                1,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', -0.0), ('i', -0.19121021032333374), ('really', -0.18888603150844574), ('didn', -0.2897638976573944), ("'", -0.006546118296682835), ('t', -0.36815303564071655), ('like', -0.15249691903591156), ('this', -0.18931037187576294), ('movie', -0.04038183391094208), ('.', -0.019403478130698204), ('some', -0.022782959043979645), ('of', -0.0), ('the', -0.023035431280732155), ('actors', -0.09489107877016068), ('were', -0.013543099164962769), ('good', -0.03236394003033638), (',', -0.00417887419462204), ('but', -0.05874660238623619), ('overall', -0.3360627293586731), ('the', -0.2185257077217102), ('movie', -0.05488923564553261), ('was', -0.6255486607551575), ('boring', -1.0), ('.', -0.03087911754846573), ('[SEP]', -0.05238119140267372)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (1.00),1,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (1.00),1,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"


# Choosing class for visualization example

In [6]:
# encode a sentence
text_batch = ["I hate that I love you."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
target_class = 0
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=11, index=target_class)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)

# get class name
class_name = classifications[target_class]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', -0.0), ('i', -0.19789810478687286), ('hate', -1.0), ('that', -0.40286505222320557), ('i', -0.1250527799129486), ('love', -0.1307108998298645), ('you', -0.05467095226049423), ('.', -6.100655809859745e-06), ('[SEP]', -0.0)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]


In [15]:
# encode a sentence
text_batch = ["I hate that I love you."]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
target_class = 1
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=11, index=target_class)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)

# get class name
class_name = classifications[target_class]
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "NEGATIVE":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,       
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)

[('[CLS]', 0.0), ('i', 0.2725590765476227), ('hate', 0.17270179092884064), ('that', 0.23211266100406647), ('i', 0.17642731964588165), ('love', 1.0), ('you', 0.2465524971485138), ('.', 0.0), ('[SEP]', 0.00015733683540020138)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,0 (0.91),1,1.00,[CLS] i hate that i love you . [SEP]
